In [2]:
%matplotlib qt5
import numpy as np
import matplotlib.pyplot as mplt
from matplotlib.animation import FuncAnimation
import pyaccel as pa
from pymodels import si

mplt.rcParams.update({'font.size': 18})

In [3]:
class Animate:
    """."""
    
    @staticmethod
    def calc_curly_h(twi, parts):
        dlt = parts[4]
        x0 = parts[0] - twi.etax * dlt
        xl0 = parts[1] - twi.etapx * dlt
        y0 = parts[2] - twi.etay * dlt
        yl0 = parts[3] - twi.etapy * dlt
        curhx = twi.betax * xl0**2 + 2*twi.alphax * xl0 * x0 + (1 + twi.alphax**2)/twi.betax * x0**2
        curhy = twi.betay * yl0**2 + 2*twi.alphay * yl0 * y0 + (1 + twi.alphay**2)/twi.betay * y0**2
        return curhx, curhy
        
    @staticmethod
    def tracking_single_particle(frames, twi, rout, every=100, title=''):
        """."""
        actx, acty = Animate.calc_curly_h(twi[0], rout[..., 0])
        dlt = rout[4, ..., 0].max()
        offx = twi.etax * dlt
        actx = actx.max()
        acty = acty.max()
        
        fig, ax = mplt.subplots(1, 1, figsize=(12, 6))
        pa.graphics.draw_lattice(mod2, gca=ax, height=5)
        ax.plot(
            twi.spos,
            (offx + np.sqrt(twi.betax*actx))*1e6,
            color='k',
            label=r'Envelope $\left(\sqrt{\epsilon \beta_x}\right)$',
        )
        if not np.isclose(dlt, 0):
            ax.plot(twi.spos, offx*1e6, '--k', label=r'Off Energy Orbit $\left(\eta_x \delta\right)$')
        ax.plot(twi.spos, (offx - np.sqrt(twi.betax*actx))*1e6, color='k')

        x = np.array(rout[0, ..., 0] * 1e6, ndmin=1)
        xl = np.array(rout[1, ..., 0] * 1e6, ndmin=1)
        spos = np.full_like(x, twi[0].spos)
        lin = ax.plot(spos, x, 'o')[0]
        ret = ax.quiver(
            spos,
            x,
            spos*0 + 1,
            xl,
            color='tab:red',
            angles='xy',
            scale_units='xy',
            scale=1,
            headlength=3,
            headaxislength=2.6,
            width=0.003,
        )

        ax.set_title(title)
        ax.set_ylabel('Horizontal Displacement [um]')
        ax.set_xlabel('Longitudinal Position [m]')
        ax.legend(loc='lower center', fontsize='small')
        fig.tight_layout()

        def update(i):
            idx = every * i
            idx = idx if idx < rout.shape[-1] else rout.shape[-1]-1
    
            x = np.array(rout[0, ..., idx] * 1e6, ndmin=1)
            xl = np.array(rout[1, ..., idx] * 1e6, ndmin=1)
            spos = np.full_like(x, twi[idx].spos)
            lin.set_data(spos, x)
            ret.set_offsets(np.array([spos, x]).T)
            ret.set_UVC(spos*0 + 1, xl)
           
            ax.relim()
            ax.autoscale_view()
            fig.tight_layout()
            return []

        return FuncAnimation(
            fig, update, fargs=(), frames=frames,
            repeat=True, repeat_delay=2000, interval=50,
            init_func=lambda: []), fig

In [4]:
mod = si.create_accelerator()

mib = pa.lattice.find_indices(mod, 'fam_name', 'mib')
mip = pa.lattice.find_indices(mod, 'fam_name', 'mip')
mod2 = mod[mib[0]:mip[0]]
mod2 = pa.lattice.refine_lattice(mod2, max_length=0.01)
twi, *_ = pa.optics.calc_twiss(mod2)
len(mod2)

2771

In [7]:
x0 = 1e-5 
xl0 = 1e-5
y0 = 1e-5 * 0
yl0 = 1e-5 * 0
dlt = 0
rin = [x0, xl0, y0, yl0, dlt, 0]
rout, *_ = pa.tracking.line_pass(mod2, rin, indices='open')

every = 1
div, mod = divmod(rout.shape[-1], every)
div += 1 if mod else 0
frames = list(range(div+1))[1:]
ann, fig = Animate.tracking_single_particle(frames, twi, rout, every=every, title='Single Partilce with Nominal Energy')

In [8]:
ann.save('single_particle_on_energy.mp4')

In [9]:
x0 = 1e-5 
xl0 = 1e-5
y0 = 1e-5 * 0
yl0 = 1e-5 * 0
dlt = 3e-4
rin = [x0, xl0, y0, yl0, dlt, 0]
rout, *_ = pa.tracking.line_pass(mod2, rin, indices='open')

every = 10
div, mod = divmod(rout.shape[-1], every)
div += 1 if mod else 0
frames = list(range(div+1))[1:]
ann, fig = Animate.tracking_single_particle(frames, twi, rout, every=every, title='Single Partilce with Offset Energy')

In [10]:
ann.save('single_particle_off_energy.mp4')

In [11]:
npart = 300
rin = np.zeros((6, npart), dtype=float)
ang = np.linspace(-1, 1, npart) * np.pi
emit = 250e-12
rin[0] = np.sqrt(emit * twi[0].betax) * np.cos(ang)
rin[1] = -np.sqrt(emit / twi[0].betax) * (twi[0].alphax * np.cos(ang) + np.sin(ang))
rout, *_ = pa.tracking.line_pass(mod2, rin, indices='open')

every = 5
div, mod = divmod(rout.shape[-1], every)
div += 1 if mod else 0
frames = list(range(div+1))[1:]
ann, fig = Animate.tracking_single_particle(frames, twi, rout, every=every, title='A Bunch of Particles')

In [12]:
ann.save('bunch_of_particles.mp4')